<a href="https://colab.research.google.com/github/rishikeshsingh1/BookMyShow/blob/master/Learning%20LLMs/3.%20StableDiffusion%20Hands-On.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login(token="")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `hf auth login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import torch
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

In [ ]:
prompt = "A futuristic city skyline at sunset"

images = pipe(prompt=prompt).images

print(len(images))
# for img in images:
#     plt.imshow(img)
#     plt.show()

#images

In [ ]:
prompt = "A futuristic city skyline at sunset"

images = pipe(prompt=prompt).images[0]

images


In [ ]:
# Get the image dimensions (height, width, and optionally color channels)
import numpy as np
numpy_image = np.array(images)
height, width, *channels = numpy_image.shape

# Print the resolution
print(f"Image Resolution: {width}x{height} pixels")

# Prompt Engineering: Improving Results

In [ ]:
# Start with the subject
prompt = "A cat"
images = pipe(prompt=prompt).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat sitting by the window"
images = pipe(prompt=prompt).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat looking right and sitting by the window"
images = pipe(prompt=prompt).images[0]
images

In [ ]:
# Add style or details
prompt = "A realistic painting of a cat sitting by the window, watercolor"
images = pipe(prompt=prompt).images[0]
images

In [ ]:
#experimenting with styles ("digital art", "watercolor", "photorealistic", etc.)
prompt = "A realistic painting of a cat sitting by the window, digital art"
images = pipe(prompt=prompt).images[0]
images

In [ ]:
'''
TRY OUT THESE PROMPTS:

prompt = "portrait of a smiling girl"
prompt = "portrait of a sad girl"
prompt = "portrait of a angry girl"
prompt = "portrait of a girl laughing"
'''

# Parameters and Customization

## Seeds: Control randomness.

In [ ]:
prompt = "A realistic painting of a cat sitting by the window"
images = pipe(prompt=prompt, seed=42).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat sitting by the window"
images = pipe(prompt=prompt, seed=42).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat sitting by the window"
images = pipe(prompt=prompt, seed=42).images[0]
images

In [ ]:
# A better prompt alongside seed can be a good way to make results consistent but not exact
prompt = "A realistic painting of a cat looking right and sitting by the left side of window"
images = pipe(prompt=prompt, seed=42).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat looking right and sitting by the left side of window"
images = pipe(prompt=prompt, seed=42).images[0]
images

In [ ]:
prompt = "A realistic painting of a cat looking right and sitting by the left side of window"
images = pipe(prompt=prompt, seed=314).images[0]
images

## Steps: More steps, better detail (but slower)

In [ ]:
prompt = "A realistic portrait of a girl"
images = pipe(prompt=prompt, num_inference_steps=1).images[0]
images

In [ ]:
prompt = "A realistic portrait of a girl"
images = pipe(prompt=prompt, num_inference_steps=10).images[0]
images

In [ ]:
prompt = "A realistic portrait of a girl"
images = pipe(prompt=prompt, num_inference_steps=30).images[0]
images

# Resolution

In [ ]:
# Set image resolution (width, height) - must be divisible by 8

#width, height = 128, 128
#width, height = 512, 512
#width, height = 1024, 512
#width, height = 512, 1024
#width, height = 1024, 1024

prompt = "Potrait of a person eating apple"
images = pipe(prompt=prompt, width=width, height=height).images[0]
images

# Logo Generation

In [ ]:
prompt = "A tech startup logo with a sleek circuit pattern forming the letter ‘T,’ sharp lines, and a metallic gradient finish, modern and minimalist, vector style, white background"
images = pipe(prompt=prompt).images[0]
images

# Mixutre of Experts

In [ ]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)

base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

# Limitations